In [1]:
import tensorflow as tf
import numpy as np
import datetime
import time

import sklearn.metrics

In [24]:
def genObs(length):
    return np.random.choice(range(3), size=length)

#very simple -- gru-rnn with 5 neurons cracks it
#def genTarget(x):
#    return ''.join([str(z) for z in x]).find('012')

def genTarget(x):
    y0 = ''.join([str(z) for z in x])
    return y0.count(y0[:2])

def genSample(num, length=20):
    x = [genObs(length=length) for _ in range(num)]
    y = [genTarget(t) for t in x]
    return np.array(x), np.array(y)

def randomBatch(tensorTuple, batchSize=64):
    ids = np.random.choice(range(tensorTuple[0].shape[0]), batchSize)
    return (x[ids,] for x in tensorTuple)

In [25]:
train_x, train_y = genSample(20000)
valid_x, valid_y = genSample(2000)
valid_x[:10], valid_y[:10]

(array([[1, 1, 1, 2, 0, 1, 2, 2, 2, 1, 2, 1, 1, 2, 2, 0, 1, 0, 1, 1],
        [2, 0, 2, 1, 0, 0, 2, 0, 0, 2, 2, 2, 1, 2, 0, 2, 1, 0, 2, 0],
        [2, 2, 1, 2, 1, 1, 0, 0, 0, 2, 1, 1, 2, 1, 2, 2, 0, 0, 0, 2],
        [2, 1, 0, 1, 0, 0, 0, 1, 1, 2, 2, 0, 0, 2, 0, 2, 1, 1, 2, 0],
        [1, 2, 1, 0, 1, 0, 2, 1, 1, 0, 2, 0, 2, 0, 2, 2, 2, 0, 2, 2],
        [1, 1, 2, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 2, 2, 1],
        [0, 0, 0, 2, 1, 1, 0, 0, 0, 2, 2, 1, 0, 2, 0, 1, 2, 0, 1, 1],
        [2, 1, 2, 0, 0, 1, 0, 2, 0, 2, 2, 2, 2, 1, 0, 1, 1, 1, 2, 2],
        [2, 0, 2, 2, 0, 2, 1, 2, 2, 2, 0, 0, 1, 2, 2, 0, 1, 0, 1, 1],
        [2, 0, 1, 0, 1, 1, 1, 0, 1, 2, 0, 2, 2, 0, 0, 0, 1, 0, 1, 2]]),
 array([3, 4, 2, 2, 1, 2, 2, 2, 4, 3]))

In [35]:
SEQ_LEN = 20
RNN_SIZE = [5,5]

InnerCell = lambda n: tf.nn.rnn_cell.GRUCell(num_units=n, activation=tf.nn.elu)
#InnerCell = lambda n: tf.nn.rnn_cell.LSTMCell(num_units=n, activation=tf.nn.relu, use_peepholes=True)

tf.reset_default_graph()

tfi_x = tf.placeholder(shape=(None, SEQ_LEN), dtype=tf.float32)
tfi_y = tf.placeholder(shape=(None), dtype=tf.float32)

tfX = tf.reshape(tfi_x, shape=(tf.shape(tfi_x)[0], tf.shape(tfi_x)[1], 1))
tfY = tf.reshape(tfi_y, shape=(tf.shape(tfi_y)[0], 1))


rnnCell = tf.nn.rnn_cell.MultiRNNCell([InnerCell(s) for s in RNN_SIZE])
#rnnCell = tf.nn.rnn_cell.GRUCell(RNN_SIZE[0])

_, tfO = tf.nn.dynamic_rnn(rnnCell, inputs=tfX, dtype=tf.float32)

tfOut = tf.layers.dense(tfO[-1], 1)

tfLoss = tf.sqrt(tf.reduce_mean(tf.square(tfY - tfOut)))
tfTrain = tf.train.AdamOptimizer(1e-3).minimize(tfLoss)

#tfOutP = tf.nn.softmax(tfOut)
tfOutR = tf.cast(tf.round(tfOut),dtype=tf.int64)

tfAccuracy = tf.reduce_mean(tf.cast(tf.equal(tf.cast(tfY, dtype=tf.int64), tf.cast(tf.round(tfOut),dtype=tf.int64)), dtype=tf.float32))
tffw = tf.summary.FileWriter('D:/Jupyter/Logs/00_A', tf.get_default_graph())
print('Graph creation complete')

Graph creation complete


In [36]:
batch_size = 1000
num_steps  = 30
num_epochs = 500

fmtstr = 'Epoch {0} ({1:1.3} sec): loss changed from {2:1.3} to {3:1.3}\t\tVL:{4:1.3f}\t\tAC:{5:1.3f}'
valid_batch = {tfi_x: valid_x, tfi_y: valid_y}
with tf.Session() as tfs:
    tfs.run(tf.global_variables_initializer())
    for i in range(num_epochs):
        mini_x, mini_y = randomBatch((train_x, train_y), batchSize=batch_size)
        train_batch = {tfi_x:mini_x, tfi_y:mini_y}
        l0 = tfLoss.eval(feed_dict=train_batch)
        t0 = time.perf_counter()
        for j in range(num_steps):
            tfTrain.run(feed_dict=train_batch)
        t1 = time.perf_counter()
        l1 = tfLoss.eval(feed_dict=train_batch)
        lv = tfLoss.eval(feed_dict=valid_batch)
        ac = tfAccuracy.eval(feed_dict=valid_batch)
        if i%10 == 0 or i == num_epochs - 1:
            print(fmtstr.format(i,t1-t0,l0,l1,lv,ac))
    valid_r = tfs.run(tfOutR, feed_dict=valid_batch)

Epoch 0 (0.579 sec): loss changed from 3.72 to 3.2		VL:3.118		AC:0.000
Epoch 10 (0.572 sec): loss changed from 1.12 to 1.12		VL:1.141		AC:0.306
Epoch 20 (0.564 sec): loss changed from 1.14 to 1.14		VL:1.145		AC:0.303
Epoch 30 (0.561 sec): loss changed from 1.16 to 1.16		VL:1.147		AC:0.303
Epoch 40 (0.558 sec): loss changed from 1.14 to 1.14		VL:1.141		AC:0.303
Epoch 50 (0.558 sec): loss changed from 1.14 to 1.14		VL:1.141		AC:0.303
Epoch 60 (0.563 sec): loss changed from 1.16 to 1.16		VL:1.141		AC:0.303
Epoch 70 (0.566 sec): loss changed from 1.11 to 1.11		VL:1.140		AC:0.303
Epoch 80 (0.584 sec): loss changed from 1.1 to 1.1		VL:1.139		AC:0.303
Epoch 90 (0.559 sec): loss changed from 1.15 to 1.15		VL:1.142		AC:0.303
Epoch 100 (0.564 sec): loss changed from 1.17 to 1.16		VL:1.141		AC:0.303
Epoch 110 (0.612 sec): loss changed from 1.17 to 1.17		VL:1.140		AC:0.303
Epoch 120 (0.588 sec): loss changed from 1.15 to 1.15		VL:1.140		AC:0.303
Epoch 130 (0.602 sec): loss changed from 1.14 to 1.1

In [34]:
#basic-rnn-5     1.140 loss, 0.303 accuracy (stopped at 200 epochs)
#gru-rnn-5       0.998 loss, 0.378 accuracy
#gru-rnn-10      0.524 loss, 0.681 accuracy (and still improving fast)
#gru-rnn-5-5     0.590 loss, 0.664 accuracy (and still improving fast)
valid_y[:10], valid_r[:10,0]

(array([3, 4, 2, 2, 1, 2, 2, 2, 4, 3]),
 array([3, 3, 2, 3, 2, 2, 3, 2, 3, 3], dtype=int64))

In [21]:
#gru-rnn, 5, elu => 0.143 loss, 0.994 accuracy
valid_y[:10], valid_r[:10,0]

AttributeError: 'str' object has no attribute 'reversed'